### CDI using simulated data and partial coherence
In this example a 2D CDI reconstruction is performed on a dataset
which presents some partial coherence (using a gaussian filter).

This is difficult because:
* of the added difficulty of simultaneously finding the object and the point-spread-function (PSF) used to model the partial coherence
* of the siemens start structure which has many voids, and thus convergence towards a correct support is complex to achieve

So you may need to restart the entire optimisation (from the CDI initialisation) before reaching a correct solution.

In [ ]:
# -*- coding: utf-8 -*-

# PyNX - Python tools for Nano-structures Crystallography
#   (c) 2017-present : ESRF-European Synchrotron Radiation Facility
#       authors:
#         Vincent Favre-Nicolin, favre@esrf.fr
%matplotlib notebook
import numpy as np
from scipy.fftpack import ifftshift, fftshift, fft2
from pynx.utils.pattern import siemens_star
from pynx.cdi import *
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

In [ ]:
# Test on a simulated pattern (2D)
n = 512

# Siemens-Star object
obj0 = siemens_star(n, nb_rays=12, r_max=60, nb_rings=3)

# Simulated iobs
iobs = abs(ifftshift(fft2(fftshift(obj0.astype(np.complex64))))) ** 2

# Convolve with Gaussian kernel
iobs = gaussian_filter(input=iobs, sigma=(0.7,0.7))

# Add Poisson noise
iobs = np.random.poisson(iobs * 1e10 / iobs.sum())
plt.imshow(np.log10(iobs))

In [ ]:
# Start from a slightly loose disc support
x, y = np.meshgrid(np.arange(-n // 2, n // 2, dtype=np.float32), np.arange(-n // 2, n // 2, dtype=np.float32))
r = np.sqrt(x ** 2 + y ** 2)
support = r < 65

mask = np.zeros_like(iobs, dtype=np.int16)
if False:
    # Mask some values in the central beam (much more difficult)
    print("Removing %6.3f%% intensity" % (iobs[255:257, 255:257].sum() / iobs.sum() * 100))
    iobs[255:257, 255:257] = 0
    mask[255:257, 255:257] = 1

cdi = CDI(fftshift(iobs), obj=None, support=fftshift(support), mask=fftshift(mask), wavelength=1e-10,
          pixel_size_detector=55e-10)

# Init real object from the chosen support
cdi = InitObjRandom(src="support", amin=0, amax=1, phirange=0) * cdi

# Initial scaling of the object [ only useful if there are masked pixels !]
cdi = ScaleObj(method='F') * cdi

In [ ]:
# Do 100 cycles of RAAR
plt.figure()
cdi = HIO(positivity=True, calc_llk=50, show_cdi=50) ** 100 * cdi

# Compute LLK
LLK() * cdi
print("LLK_n = %8.3f" % (cdi.get_llk(noise='poisson')))

cdi = ShowCDI(fig_num=1) * cdi

In [ ]:
plt.figure()
sup = SupportUpdate(threshold_relative=0.4, smooth_width=(2.0, 0.5, 800), force_shrink=False)

cdi = (sup * ER(positivity=True, calc_llk=40, show_cdi=40, fig_num=1) ** 5 * 
       HIO(positivity=True,calc_llk=40, show_cdi=40) ** 40) ** 5 * cdi


In [ ]:
# Now Estimate the PSF and update it regularly
cdi = InitPSF(model="gaussian", fwhm=0.5) * cdi


sup = SupportUpdate(threshold_relative=0.35, smooth_width=(1.0, 0.5, 800), force_shrink=False)
er = ER(positivity=True, calc_llk=40, show_cdi=40, fig_num=1, update_psf=20)
raar = RAAR(positivity=True, calc_llk=40, show_cdi=40, update_psf=20)
hio = HIO(positivity=True, calc_llk=40, show_cdi=40, update_psf=20)

plt.figure()
cdi = (sup * er ** 5 * hio ** 40) ** 5 * cdi


In [ ]:
# Release GPU memory
cdi = FreePU() * cdi